In [12]:
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver

In [13]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path='C:/Users/defri/chromedriver/chromedriver.exe', options=options) 
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword + '&locT=N&locId=113&locKeyword=Indonesia&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []
    
    start = time.time()
    print("Gathering Information")

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)
        
        try:
            driver.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
        except NoSuchElementException:
            pass
        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            #print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs))) #print progress through job searching
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    print(f"Scraping Completed for {num_jobs} jobs")
    print(f"Time Required : {time.time() - start} seconds")
        
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.
    

In [15]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 150, False)
df

Gathering Information
Scraping Completed for 150 jobs
Time Required : 623.3329181671143 seconds


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Big Data & Analytics Consultant,-1,Are you passionate about analytics? Would you ...,3.9,Amazon\n3.9,Jakarta,10000+ Employees,1994,Company - Public,Internet,Information Technology,$10+ billion (USD)
1,Data Scientist - QuantumBlack,-1,QUALIFICATIONS\nBachelor or Masters or PhD lev...,4.4,McKinsey & Company\n4.4,Jakarta,10000+ Employees,1926,Company - Private,Consulting,Business Services,Unknown / Non-Applicable
2,IT Security Data Scientist,-1,Job Title:\nIT Security Data Scientist\n\nJob ...,3.9,Nomura Holdings\n3.9,Emea,10000+ Employees,1925,Company - Public,Investment Banking & Asset Management,Finance,$10+ billion (USD)
3,Data Scientist - Risk Modelling Specialist,-1,Company Description\n\nCermati is a financial ...,4.4,Cermati.com\n4.4,Jakarta,1 to 50 Employees,-1,Company - Private,Internet,Information Technology,Less than $1 million (USD)
4,Data Scientist,-1,HappyFresh is Southeast Asia's #1 Online Groce...,4.3,HappyFresh\n4.3,Jakarta,201 to 500 Employees,2014,Company - Private,Internet,Information Technology,Unknown / Non-Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...
145,DATA ANALYST(FASHION RETAIL),-1,Role and Responsibilities:\n\nAnalyze the bran...,-1,PT THE CLOTHES,Jakarta,-1,-1,-1,-1,-1,-1
146,Data Engineer,-1,"Lokasi: Jakarta, Jakarta\nFull time\nMekari is...",4.0,Jurnal Consulting\n4.0,Jakarta,51 to 200 Employees,-1,Company - Private,-1,-1,Less than $1 million (USD)
147,Data Analyst & Business Intelligence,-1,Bekerja sama dengan Business User dalam meranc...,-1,PUNINAR LOGISTICS,Jakarta,-1,-1,-1,-1,-1,-1
148,Data Engineer,-1,"Responsibilities :\n\nPresentation, create pro...",4.0,Computrade Technology\n4.0,Jakarta,201 to 500 Employees,-1,Company - Private,-1,-1,Less than $1 million (USD)


In [16]:
#Save the file to csv
df.to_csv('glassdoor job posting.csv', index=False, encoding='utf-8')